# Output Fixing Parser


`OutputFixingParser` trong LangChain cung cấp một cơ chế tự động để sửa các lỗi có thể xảy ra trong quá trình phân tích cú pháp đầu ra. Parser này được thiết kế để bao bọc xung quanh một parser khác, chẳng hạn như `PydanticOutputParser`, và can thiệp khi parser cơ bản gặp phải các đầu ra bị lỗi định dạng hoặc không tuân thủ định dạng mong đợi. Nó đạt được điều này bằng cách tận dụng các lệnh gọi LLM bổ sung để sửa lỗi và đảm bảo định dạng phù hợp.

Về cốt lõi, `OutputFixingParser` giải quyết các tình huống mà đầu ra ban đầu không tuân thủ một lược đồ được xác định trước. Nếu một vấn đề như vậy phát sinh, parser tự động phát hiện các lỗi định dạng và gửi một yêu cầu mới đến mô hình, bao gồm các hướng dẫn cụ thể để sửa vấn đề. Những hướng dẫn này làm nổi bật các khu vực có vấn đề và cung cấp các hướng dẫn rõ ràng để tái cấu trúc dữ liệu theo định dạng chính xác.

Chức năng này đặc biệt hữu ích trong các tình huống mà việc tuân thủ nghiêm ngặt một lược đồ là rất quan trọng. Ví dụ: khi sử dụng `PydanticOutputParser` để tạo ra các đầu ra tuân thủ một lược đồ dữ liệu cụ thể, các vấn đề như thiếu trường hoặc kiểu dữ liệu không chính xác có thể xảy ra.

- `OutputFixingParser` can thiệp như sau:

1.  **Phát hiện lỗi (Error Detection)**: Nó nhận ra rằng đầu ra không đáp ứng các yêu cầu lược đồ.
2.  **Sửa lỗi (Error Correction)**: Nó tạo ra một yêu cầu tiếp theo đến LLM với các hướng dẫn rõ ràng để giải quyết các vấn đề.
3.  **Đầu ra được định dạng lại với các hướng dẫn cụ thể (Reformatted Output with Specific Instructions)**: `OutputFixingParser` đảm bảo rằng các hướng dẫn sửa lỗi xác định chính xác các lỗi, chẳng hạn như thiếu trường hoặc kiểu dữ liệu không chính xác. Các hướng dẫn hướng dẫn LLM định dạng lại đầu ra để đáp ứng các yêu cầu lược đồ một cách chính xác.

----

Ví dụ thực tế:

Giả sử bạn đang sử dụng `PydanticOutputParser` để thực thi một lược đồ yêu cầu các trường cụ thể như `name` (string), `age` (integer) và `email` (string). Nếu LLM tạo ra một đầu ra mà trường `age` bị thiếu hoặc trường `email` không phải là một chuỗi hợp lệ, `OutputFixingParser` sẽ tự động can thiệp. Nó sẽ đưa ra một yêu cầu mới đến LLM với các hướng dẫn chi tiết như:

-   "Đầu ra bị thiếu trường `age`. Thêm một giá trị số nguyên thích hợp cho `age`."
-   "Trường `email` chứa một định dạng không hợp lệ. Sửa nó để phù hợp với một chuỗi email hợp lệ."

Quá trình lặp đi lặp lại này đảm bảo đầu ra cuối cùng tuân thủ lược đồ được chỉ định mà không cần can thiệp thủ công.

----

Lợi ích chính:

-   **Khôi phục lỗi (Error Recovery)**: Tự động xử lý các đầu ra bị lỗi định dạng mà không cần đầu vào của người dùng.
-   **Tăng cường độ chính xác (Enhanced Accuracy)**: Đảm bảo đầu ra tuân thủ các lược đồ được xác định trước, giảm nguy cơ không nhất quán.
-   **Quy trình làm việc được sắp xếp hợp lý (Streamlined Workflow)**: Giảm thiểu nhu cầu sửa chữa thủ công, tiết kiệm thời gian và cải thiện hiệu quả.

----

Các bước triển khai:

Để sử dụng `OutputFixingParser` một cách hiệu quả, hãy làm theo các bước sau:

1.  **Bao bọc một Parser (Wrap a Parser)**: Khởi tạo `OutputFixingParser` với một parser khác, chẳng hạn như `PydanticOutputParser`, làm cơ sở của nó.
2.  **Xác định lược đồ (Define the Schema)**: Chỉ định lược đồ hoặc định dạng mà đầu ra phải tuân thủ.
3.  **Cho phép sửa lỗi (Enable Error Correction)**: Cho phép `OutputFixingParser` tự động phát hiện và sửa lỗi thông qua các lệnh gọi LLM bổ sung, đảm bảo rằng các hướng dẫn sửa lỗi xác định và giải quyết chính xác các vấn đề để định dạng lại chính xác.

Bằng cách tích hợp `OutputFixingParser` vào quy trình làm việc của bạn, bạn có thể đảm bảo xử lý lỗi mạnh mẽ và duy trì chất lượng đầu ra nhất quán trong các ứng dụng LangChain của mình.


In [1]:
# Setup environment
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

load_dotenv(override=True, dotenv_path="../.env")

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.0)

## Define Data Model and Set Up PydanticOutputParser

- The Actor class is defined using the Pydantic model, where name and film_names are fields representing the actor's name and a list of films they starred in.
- The `PydanticOutputParser` is used to parse outputs into an Actor object.

In [2]:
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from typing import List


# Define the Actor class using Pydantic
class Actor(BaseModel):
    name: str = Field(description="name of an actor")
    film_names: List[str] = Field(description="list of names of films they starred in")


# A query to generate the filmography for a random actor
actor_query = "Generate the filmography for a random actor."

# Use PydanticOutputParser to parse the output into an Actor object
parser = PydanticOutputParser(pydantic_object=Actor)

### Attempt to Parse Misformatted Input Data

- The misformatted variable contains an incorrectly formatted string, which does not match the expected structure (using ' instead of ").
- Calling `parser.parse()` will result in an error because of the format mismatch.

In [3]:
# Intentionally input misformatted data
misformatted = "{'name': 'Tom Hanks', 'film_names': ['Forrest Gump']}"
parser.parse(misformatted)

# An error will be printed because the data is incorrectly formatted

OutputParserException: Invalid json output: {'name': 'Tom Hanks', 'film_names': ['Forrest Gump']}
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE 

## Using OutputFixingParser to Correct Incorrect Formatting
### Set Up OutputFixingParser to Automatically Correct the Error
- `OutputFixingParser` wraps around the existing `PydanticOutputParser` and automatically fixes errors by making additional calls to the LLM.
- The `from_llm()` method connects `OutputFixingParser` with `ChatOpenAI` to correct the formatting issues in the output.

In [4]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain.output_parsers import OutputFixingParser

# Define a custom prompt to provide the fixing instructions
fixing_prompt = PromptTemplate(
    template=(
        "The following JSON is incorrectly formatted or incomplete: {completion}\n"
    ),
    input_variables=[
        "completion",
    ],
)

# Use OutputFixingParser to automatically fix the error
new_parser = OutputFixingParser.from_llm(
    parser=parser, llm=ChatOpenAI(model_name="gpt-3.5-turbo"), prompt=fixing_prompt
)

### Parse the Misformatted Output Using OutputFixingParser
- The `new_parser.parse()` method is used to parse the misformatted data. OutputFixingParser will correct the errors in the data and generate a valid Actor object.

In [6]:
misformatted

"{'name': 'Tom Hanks', 'film_names': ['Forrest Gump']}"

In [7]:
actor = new_parser.parse(misformatted)

OutputParserException: Failed to parse Actor from completion null. Got: 1 validation error for Actor
  Input should be a valid dictionary or instance of Actor [type=model_type, input_value=None, input_type=NoneType]
    For further information visit https://errors.pydantic.dev/2.10/v/model_type
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE 